In [1]:
# Import the required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import sqlite3
import hvplot.pandas
import numpy as np


In [2]:
connection = sqlite3.connect('../data/db.sqlite')

# Import DB into pandas dataframe
df = pd.read_sql_query("SELECT * FROM ratings", connection)

connection.close()

df.head()

,Rating Agency,Corporation,Rating,Rating Date,CIK,Binary Rating,SIC Code,Sector,Ticker,Current Ratio,...,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover,ROE - Return On Equity,Return On Tangible Equity,ROA - Return On Assets,ROI - Return On Investment,Operating Cash Flow Per Share,Free Cash Flow Per Share
0,Standard & Poor's Ratings Services,American States Water Co.,A-,2010-07-30,1056903,1,4941.0,Utils,AWR,1.1507,...,28.9834,13.6093,8.3224,0.3173,8.1724,8.1978,2.6385,4.4530,1.9957,-0.1333
1,Standard & Poor's Ratings Services,Automatic Data Processing Inc.,AAA,2010-09-16,8670,1,7374.0,BusEq,ADP,1.1129,...,23.9379,20.8699,13.5690,0.3324,22.0354,47.2858,4.4944,21.8765,0.2501,0.3132
2,Standard & Poor's Ratings Services,Avnet Inc.,BBB-,2010-11-23,8858,1,5065.0,Shops,AVT,1.9276,...,3.6338,3.0536,2.1418,2.4620,13.6376,16.7991,5.2731,9.6494,-7.6079,-7.3231
3,Standard & Poor's Ratings Services,California Water Service Co.,AA-,2010-06-29,1035201,1,4941.0,Utils,CWT,0.8358,...,27.9377,15.1135,9.0246,0.2946,9.6412,9.7015,2.6583,5.1018,1.7438,-0.8999
4,Standard & Poor's Ratings Services,Cardinal Health Inc.,A,2010-07-14,721371,1,5122.0,Shops,CAH,1.2931,...,1.5847,1.2304,0.6518,4.9276,11.1256,19.4184,2.9364,8.1844,1.9725,2.4174


In [3]:
reduced_df = df.drop(columns=['CIK', 'Ticker', 'Rating Date', 'SIC Code'])
reduced_df.head()

,Rating Agency,Corporation,Rating,Binary Rating,Sector,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,...,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover,ROE - Return On Equity,Return On Tangible Equity,ROA - Return On Assets,ROI - Return On Investment,Operating Cash Flow Per Share,Free Cash Flow Per Share
0,Standard & Poor's Ratings Services,American States Water Co.,A-,1,Utils,1.1507,0.4551,0.8847,77.6230,19.4839,...,28.9834,13.6093,8.3224,0.3173,8.1724,8.1978,2.6385,4.4530,1.9957,-0.1333
1,Standard & Poor's Ratings Services,Automatic Data Processing Inc.,AAA,1,BusEq,1.1129,0.0072,0.0073,43.6619,19.8327,...,23.9379,20.8699,13.5690,0.3324,22.0354,47.2858,4.4944,21.8765,0.2501,0.3132
2,Standard & Poor's Ratings Services,Avnet Inc.,BBB-,1,Shops,1.9276,0.2924,0.4255,11.9008,3.3173,...,3.6338,3.0536,2.1418,2.4620,13.6376,16.7991,5.2731,9.6494,-7.6079,-7.3231
3,Standard & Poor's Ratings Services,California Water Service Co.,AA-,1,Utils,0.8358,0.4708,0.9491,64.5096,18.4549,...,27.9377,15.1135,9.0246,0.2946,9.6412,9.7015,2.6583,5.1018,1.7438,-0.8999
4,Standard & Poor's Ratings Services,Cardinal Health Inc.,A,1,Shops,1.2931,0.2644,0.4036,3.8385,1.3269,...,1.5847,1.2304,0.6518,4.9276,11.1256,19.4184,2.9364,8.1844,1.9725,2.4174


In [4]:
reduced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7805 entries, 0 to 7804
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Rating Agency                  7805 non-null   object 
 1   Corporation                    7805 non-null   object 
 2   Rating                         7805 non-null   object 
 3   Binary Rating                  7805 non-null   int64  
 4   Sector                         7805 non-null   object 
 5   Current Ratio                  7805 non-null   float64
 6   Long-term Debt / Capital       7805 non-null   float64
 7   Debt/Equity Ratio              7805 non-null   float64
 8   Gross Margin                   7805 non-null   float64
 9   Operating Margin               7805 non-null   float64
 10  EBIT Margin                    7805 non-null   float64
 11  EBITDA Margin                  7805 non-null   float64
 12  Pre-Tax Profit Margin          7805 non-null   f

In [5]:
reduced_df.shape

(7805, 21)

### Step 2: Scale the `df_stocks` DataFrame and create a new DataFrame that contains the scaled data. 

In [6]:
# Scale price data, return, and variance values
data_scaled = StandardScaler().fit_transform(
    reduced_df[["Current Ratio", "Long-term Debt / Capital", "Debt/Equity Ratio", "Gross Margin", "Operating Margin", "EBIT Margin", "EBITDA Margin", \
        "Pre-Tax Profit Margin", "Net Profit Margin", "Asset Turnover", "ROE - Return On Equity", "Return On Tangible Equity", "ROA - Return On Assets", \
        "ROI - Return On Investment", "Operating Cash Flow Per Share", "Free Cash Flow Per Share"]]
)

In [7]:
# Create a DataFrame with the scaled data
df_scaled = pd.DataFrame(
    data_scaled,
    columns=["Current Ratio", "Long-term Debt / Capital", "Debt/Equity Ratio", "Gross Margin", "Operating Margin", "EBIT Margin", "EBITDA Margin", \
        "Pre-Tax Profit Margin", "Net Profit Margin", "Asset Turnover", "ROE - Return On Equity", "Return On Tangible Equity", "ROA - Return On Assets", \
        "ROI - Return On Investment", "Operating Cash Flow Per Share", "Free Cash Flow Per Share"]
)

# Copy the tickers names from the original data
#df_scaled["Corporation"] = df['Corporation']

# shift column 'Name' to first position 
first_column = df.pop('Corporation') 
  
# insert column using insert(position,column_name, 
# first_column) function 
df_scaled.insert(0, 'Corporation', first_column) 

# Set the Ticker column as index
# df_scaled = df_scaled.set_index("Corporation")

# Display sample data
df_scaled.head()

,Corporation,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover,ROE - Return On Equity,Return On Tangible Equity,ROA - Return On Assets,ROI - Return On Investment,Operating Cash Flow Per Share,Free Cash Flow Per Share
0,American States Water Co.,-0.403644,0.000873,0.020530,1.482239,0.312253,0.310067,0.365559,0.177621,0.096734,-0.764973,-0.033733,-0.028709,-0.209897,-0.107590,0.250559,-0.034852
1,Automatic Data Processing Inc.,-0.423316,-0.169496,-0.004936,0.051726,0.325949,0.323755,0.158597,0.438188,0.314851,-0.743282,0.026390,0.055768,-0.008809,0.607060,-0.037958,0.026707
2,Avnet Inc.,0.000670,-0.061013,0.007202,-1.286119,-0.322542,-0.324358,-0.674261,-0.201199,-0.160213,2.315788,-0.010031,-0.010120,0.075563,0.105548,-1.336750,-1.026112
3,California Water Service Co.,-0.567524,0.006845,0.022399,0.929875,0.271849,0.269686,0.322665,0.231604,0.125926,-0.797580,-0.027363,-0.025459,-0.207752,-0.080978,0.208925,-0.140544
4,Cardinal Health Inc.,-0.329536,-0.071664,0.006567,-1.625720,-0.400697,-0.402467,-0.758313,-0.266630,-0.222157,5.857507,-0.020925,-0.004459,-0.177620,0.045459,0.246725,0.316813


In [8]:
# Encode the "EnergyType" column to variables to categorize oil versus non-oil firms. 
dummies = pd.get_dummies(df, columns=['Rating', 'Sector'])
dummies.head()

,Rating Agency,Rating Date,CIK,Binary Rating,SIC Code,Ticker,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,...,Sector_Durbl,Sector_Enrgy,Sector_Hlth,Sector_Manuf,Sector_Money,Sector_NoDur,Sector_Other,Sector_Shops,Sector_Telcm,Sector_Utils
0,Standard & Poor's Ratings Services,2010-07-30,1056903,1,4941.0,AWR,1.1507,0.4551,0.8847,77.6230,...,False,False,False,False,False,False,False,False,False,True
1,Standard & Poor's Ratings Services,2010-09-16,8670,1,7374.0,ADP,1.1129,0.0072,0.0073,43.6619,...,False,False,False,False,False,False,False,False,False,False
2,Standard & Poor's Ratings Services,2010-11-23,8858,1,5065.0,AVT,1.9276,0.2924,0.4255,11.9008,...,False,False,False,False,False,False,False,True,False,False
3,Standard & Poor's Ratings Services,2010-06-29,1035201,1,4941.0,CWT,0.8358,0.4708,0.9491,64.5096,...,False,False,False,False,False,False,False,False,False,True
4,Standard & Poor's Ratings Services,2010-07-14,721371,1,5122.0,CAH,1.2931,0.2644,0.4036,3.8385,...,False,False,False,False,False,False,False,True,False,False


In [9]:
# Concatenate the "EnergyType" variables with the scaled data DataFrame.
df_scaled = pd.concat([df_scaled, dummies], axis=1)

# Display the sample data
df_scaled.head()

,Corporation,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,...,Sector_Durbl,Sector_Enrgy,Sector_Hlth,Sector_Manuf,Sector_Money,Sector_NoDur,Sector_Other,Sector_Shops,Sector_Telcm,Sector_Utils
0,American States Water Co.,-0.403644,0.000873,0.020530,1.482239,0.312253,0.310067,0.365559,0.177621,0.096734,...,False,False,False,False,False,False,False,False,False,True
1,Automatic Data Processing Inc.,-0.423316,-0.169496,-0.004936,0.051726,0.325949,0.323755,0.158597,0.438188,0.314851,...,False,False,False,False,False,False,False,False,False,False
2,Avnet Inc.,0.000670,-0.061013,0.007202,-1.286119,-0.322542,-0.324358,-0.674261,-0.201199,-0.160213,...,False,False,False,False,False,False,False,True,False,False
3,California Water Service Co.,-0.567524,0.006845,0.022399,0.929875,0.271849,0.269686,0.322665,0.231604,0.125926,...,False,False,False,False,False,False,False,False,False,True
4,Cardinal Health Inc.,-0.329536,-0.071664,0.006567,-1.625720,-0.400697,-0.402467,-0.758313,-0.266630,-0.222157,...,False,False,False,False,False,False,False,True,False,False


In [10]:
# Create a a list to store inertia values
inertia = []

# Create a a list to store the values of k
k = list(range(1, 11))

# Create copy of df without corporation field
df_copy = df_scaled
df_copy = df_copy.drop(columns=['Corporation', 'Ticker', 'Rating Agency', 'Rating Date'])
df_copy.head()


,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover,...,Sector_Durbl,Sector_Enrgy,Sector_Hlth,Sector_Manuf,Sector_Money,Sector_NoDur,Sector_Other,Sector_Shops,Sector_Telcm,Sector_Utils
0,-0.403644,0.000873,0.020530,1.482239,0.312253,0.310067,0.365559,0.177621,0.096734,-0.764973,...,False,False,False,False,False,False,False,False,False,True
1,-0.423316,-0.169496,-0.004936,0.051726,0.325949,0.323755,0.158597,0.438188,0.314851,-0.743282,...,False,False,False,False,False,False,False,False,False,False
2,0.000670,-0.061013,0.007202,-1.286119,-0.322542,-0.324358,-0.674261,-0.201199,-0.160213,2.315788,...,False,False,False,False,False,False,False,True,False,False
3,-0.567524,0.006845,0.022399,0.929875,0.271849,0.269686,0.322665,0.231604,0.125926,-0.797580,...,False,False,False,False,False,False,False,False,False,True
4,-0.329536,-0.071664,0.006567,-1.625720,-0.400697,-0.402467,-0.758313,-0.266630,-0.222157,5.857507,...,False,False,False,False,False,False,False,True,False,False


In [11]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7805 entries, 0 to 7804
Data columns (total 70 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Current Ratio                  7805 non-null   float64
 1   Long-term Debt / Capital       7805 non-null   float64
 2   Debt/Equity Ratio              7805 non-null   float64
 3   Gross Margin                   7805 non-null   float64
 4   Operating Margin               7805 non-null   float64
 5   EBIT Margin                    7805 non-null   float64
 6   EBITDA Margin                  7805 non-null   float64
 7   Pre-Tax Profit Margin          7805 non-null   float64
 8   Net Profit Margin              7805 non-null   float64
 9   Asset Turnover                 7805 non-null   float64
 10  ROE - Return On Equity         7805 non-null   float64
 11  Return On Tangible Equity      7805 non-null   float64
 12  ROA - Return On Assets         7805 non-null   f

In [12]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
for i in k:
    k_model = KMeans(n_clusters=i, random_state=2)
    k_model.fit(df_copy)
    inertia.append(k_model.inertia_)

# Create a Dictionary that holds the list values for k and inertia
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame using the elbow_data Dictionary
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,2.064657e+15
1,2,3.582538e+14
2,3,1.117985e+14
3,4,5.862009e+13
4,5,4.133709e+13


In [13]:
# Plot Elbow Curve
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

In [14]:
# Initialize the K-Means model with n_clusters=3
model = KMeans(n_clusters=3)

In [15]:
# Fit the model for the df_stocks_scaled DataFrame
model.fit(df_copy)

KMeans(n_clusters=3)

In [16]:
# Predict the model segments (clusters)
clusters = model.predict(df_copy)

# View the stock segments
print(clusters)

[2 1 1 ... 2 2 2]


In [17]:
unique = np.unique(clusters)
unique

array([0, 1, 2])

In [18]:
# Create a new column in the DataFrame with the predicted clusters
df_scaled["Clusters"] = clusters

# Review the DataFrame
df_scaled.head()

,Corporation,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,...,Sector_Enrgy,Sector_Hlth,Sector_Manuf,Sector_Money,Sector_NoDur,Sector_Other,Sector_Shops,Sector_Telcm,Sector_Utils,Clusters
0,American States Water Co.,-0.403644,0.000873,0.020530,1.482239,0.312253,0.310067,0.365559,0.177621,0.096734,...,False,False,False,False,False,False,False,False,True,2
1,Automatic Data Processing Inc.,-0.423316,-0.169496,-0.004936,0.051726,0.325949,0.323755,0.158597,0.438188,0.314851,...,False,False,False,False,False,False,False,False,False,1
2,Avnet Inc.,0.000670,-0.061013,0.007202,-1.286119,-0.322542,-0.324358,-0.674261,-0.201199,-0.160213,...,False,False,False,False,False,False,True,False,False,1
3,California Water Service Co.,-0.567524,0.006845,0.022399,0.929875,0.271849,0.269686,0.322665,0.231604,0.125926,...,False,False,False,False,False,False,False,False,True,2
4,Cardinal Health Inc.,-0.329536,-0.071664,0.006567,-1.625720,-0.400697,-0.402467,-0.758313,-0.266630,-0.222157,...,False,False,False,False,False,False,True,False,False,2


In [19]:
# Create the PCA model instance where n_components=2
pca = PCA(n_components=2)

In [20]:
# Fit the df_stocks_scaled data to the PCA
pca_data = pca.fit_transform(df_copy)

# Review the first five rose of the PCA data
# using bracket notation ([0:5])
pca_data[:5]

array([[ 319741.64171363,     801.32823232],
       [-728490.65730188,    3519.62047625],
       [-728303.28509063,    1210.2331348 ],
       [ 298039.64247256,     807.23394553],
       [ -15790.2966805 ,    1073.49894357]])

In [21]:
# Calculate the explained variance
pca.explained_variance_ratio_

array([9.99985814e-01, 1.31574379e-05])

In [22]:
# Creating a DataFrame with the PCA data
df_pca = pd.DataFrame(pca_data, columns=["PC1", "PC2"])

# Copy the  names from the original data
# df_pca.insert(0, 'Corporation', first_column)

# Review the DataFrame
df_pca.head()

,PC1,PC2
0,319741.641714,801.328232
1,-728490.657302,3519.620476
2,-728303.285091,1210.233135
3,298039.642473,807.233946
4,-15790.296680,1073.498944


In [23]:
# Initialize the K-Means model with n_clusters=3
model = KMeans(n_clusters=3)

# Fit the model for the df_stocks_pca DataFrame
model.fit(df_pca)

# Predict the model segments (clusters)
clusters2 = model.predict(df_pca)

# Print the stock segments
print(clusters2)

[0 1 1 ... 0 0 0]


In [24]:
# Create a copy of the df_stocks_pca DataFrame and name it as df_stocks_pca_predictions
df_pca_predictions = df_pca.copy()

# Create a new column in the DataFrame with the predicted clusters
df_pca_predictions["Cluster"] = clusters2

# Review the DataFrame
df_pca_predictions.head()

,PC1,PC2,Cluster
0,319741.641714,801.328232,0
1,-728490.657302,3519.620476,1
2,-728303.285091,1210.233135,1
3,298039.642473,807.233946,0
4,-15790.296680,1073.498944,0


In [25]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1, 11))

In [26]:
# Create an empy list to store the inertia values
inertia = []

In [27]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_stocks_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_pca_predictions)
    inertia.append(model.inertia_)

In [28]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)

In [29]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(x="k", y="inertia", title="Elbow Curve Using PCA Data", xticks=k)
elbow_plot_pca

:Curve   [k]   (inertia)

In [30]:
df_copy.columns

Index(['Current Ratio', 'Long-term Debt / Capital', 'Debt/Equity Ratio',
       'Gross Margin', 'Operating Margin', 'EBIT Margin', 'EBITDA Margin',
       'Pre-Tax Profit Margin', 'Net Profit Margin', 'Asset Turnover',
       'ROE - Return On Equity', 'Return On Tangible Equity',
       'ROA - Return On Assets', 'ROI - Return On Investment',
       'Operating Cash Flow Per Share', 'Free Cash Flow Per Share', 'CIK',
       'Binary Rating', 'SIC Code', 'Current Ratio',
       'Long-term Debt / Capital', 'Debt/Equity Ratio', 'Gross Margin',
       'Operating Margin', 'EBIT Margin', 'EBITDA Margin',
       'Pre-Tax Profit Margin', 'Net Profit Margin', 'Asset Turnover',
       'ROE - Return On Equity', 'Return On Tangible Equity',
       'ROA - Return On Assets', 'ROI - Return On Investment',
       'Operating Cash Flow Per Share', 'Free Cash Flow Per Share', 'Rating_A',
       'Rating_A+', 'Rating_A-', 'Rating_AA', 'Rating_AA+', 'Rating_AA-',
       'Rating_AAA', 'Rating_B', 'Rating_B+', '

In [31]:
df.columns

Index(['Rating Agency', 'Rating', 'Rating Date', 'CIK', 'Binary Rating',
       'SIC Code', 'Sector', 'Ticker', 'Current Ratio',
       'Long-term Debt / Capital', 'Debt/Equity Ratio', 'Gross Margin',
       'Operating Margin', 'EBIT Margin', 'EBITDA Margin',
       'Pre-Tax Profit Margin', 'Net Profit Margin', 'Asset Turnover',
       'ROE - Return On Equity', 'Return On Tangible Equity',
       'ROA - Return On Assets', 'ROI - Return On Investment',
       'Operating Cash Flow Per Share', 'Free Cash Flow Per Share'],
      dtype='object')

In [32]:
# shift column 'Name' to first position 
#first_column = df.pop('Corporation') 
  
# insert column using insert(position,column_name, 
# first_column) function 
df_copy.insert(0, 'Corporation', first_column) 
df_copy.head()

,Corporation,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,...,Sector_Durbl,Sector_Enrgy,Sector_Hlth,Sector_Manuf,Sector_Money,Sector_NoDur,Sector_Other,Sector_Shops,Sector_Telcm,Sector_Utils
0,American States Water Co.,-0.403644,0.000873,0.020530,1.482239,0.312253,0.310067,0.365559,0.177621,0.096734,...,False,False,False,False,False,False,False,False,False,True
1,Automatic Data Processing Inc.,-0.423316,-0.169496,-0.004936,0.051726,0.325949,0.323755,0.158597,0.438188,0.314851,...,False,False,False,False,False,False,False,False,False,False
2,Avnet Inc.,0.000670,-0.061013,0.007202,-1.286119,-0.322542,-0.324358,-0.674261,-0.201199,-0.160213,...,False,False,False,False,False,False,False,True,False,False
3,California Water Service Co.,-0.567524,0.006845,0.022399,0.929875,0.271849,0.269686,0.322665,0.231604,0.125926,...,False,False,False,False,False,False,False,False,False,True
4,Cardinal Health Inc.,-0.329536,-0.071664,0.006567,-1.625720,-0.400697,-0.402467,-0.758313,-0.266630,-0.222157,...,False,False,False,False,False,False,False,True,False,False


In [33]:
agency = df.pop('Rating Agency')

df_copy.insert(1, 'Rating Agency', agency)
df_copy.head()

,Corporation,Rating Agency,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,...,Sector_Durbl,Sector_Enrgy,Sector_Hlth,Sector_Manuf,Sector_Money,Sector_NoDur,Sector_Other,Sector_Shops,Sector_Telcm,Sector_Utils
0,American States Water Co.,Standard & Poor's Ratings Services,-0.403644,0.000873,0.020530,1.482239,0.312253,0.310067,0.365559,0.177621,...,False,False,False,False,False,False,False,False,False,True
1,Automatic Data Processing Inc.,Standard & Poor's Ratings Services,-0.423316,-0.169496,-0.004936,0.051726,0.325949,0.323755,0.158597,0.438188,...,False,False,False,False,False,False,False,False,False,False
2,Avnet Inc.,Standard & Poor's Ratings Services,0.000670,-0.061013,0.007202,-1.286119,-0.322542,-0.324358,-0.674261,-0.201199,...,False,False,False,False,False,False,False,True,False,False
3,California Water Service Co.,Standard & Poor's Ratings Services,-0.567524,0.006845,0.022399,0.929875,0.271849,0.269686,0.322665,0.231604,...,False,False,False,False,False,False,False,False,False,True
4,Cardinal Health Inc.,Standard & Poor's Ratings Services,-0.329536,-0.071664,0.006567,-1.625720,-0.400697,-0.402467,-0.758313,-0.266630,...,False,False,False,False,False,False,False,True,False,False


In [34]:
rating = df.pop('Rating')

df_copy.insert(2, 'Rating', rating)
df_copy.head()

,Corporation,Rating Agency,Rating,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,...,Sector_Durbl,Sector_Enrgy,Sector_Hlth,Sector_Manuf,Sector_Money,Sector_NoDur,Sector_Other,Sector_Shops,Sector_Telcm,Sector_Utils
0,American States Water Co.,Standard & Poor's Ratings Services,A-,-0.403644,0.000873,0.020530,1.482239,0.312253,0.310067,0.365559,...,False,False,False,False,False,False,False,False,False,True
1,Automatic Data Processing Inc.,Standard & Poor's Ratings Services,AAA,-0.423316,-0.169496,-0.004936,0.051726,0.325949,0.323755,0.158597,...,False,False,False,False,False,False,False,False,False,False
2,Avnet Inc.,Standard & Poor's Ratings Services,BBB-,0.000670,-0.061013,0.007202,-1.286119,-0.322542,-0.324358,-0.674261,...,False,False,False,False,False,False,False,True,False,False
3,California Water Service Co.,Standard & Poor's Ratings Services,AA-,-0.567524,0.006845,0.022399,0.929875,0.271849,0.269686,0.322665,...,False,False,False,False,False,False,False,False,False,True
4,Cardinal Health Inc.,Standard & Poor's Ratings Services,A,-0.329536,-0.071664,0.006567,-1.625720,-0.400697,-0.402467,-0.758313,...,False,False,False,False,False,False,False,True,False,False


In [35]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7805 entries, 0 to 7804
Data columns (total 73 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Corporation                    7805 non-null   object 
 1   Rating Agency                  7805 non-null   object 
 2   Rating                         7805 non-null   object 
 3   Current Ratio                  7805 non-null   float64
 4   Long-term Debt / Capital       7805 non-null   float64
 5   Debt/Equity Ratio              7805 non-null   float64
 6   Gross Margin                   7805 non-null   float64
 7   Operating Margin               7805 non-null   float64
 8   EBIT Margin                    7805 non-null   float64
 9   EBITDA Margin                  7805 non-null   float64
 10  Pre-Tax Profit Margin          7805 non-null   float64
 11  Net Profit Margin              7805 non-null   float64
 12  Asset Turnover                 7805 non-null   f

In [36]:
duplicate_columns = [
    'Current Ratio',
    'Long-term Debt / Capital',
    'Debt/Equity Ratio',
    'Gross Margin',
    'Operating Margin',
    'EBIT Margin',
    'EBITDA Margin',
    'Pre-Tax Profit Margin',
    'Net Profit Margin',
    'Asset Turnover',
    'ROE - Return On Equity',
    'Return On Tangible Equity',
    'ROA - Return On Assets',
    'ROI - Return On Investment',
    'Operating Cash Flow Per Share',
    'Free Cash Flow Per Share'
]

df_copy.drop(duplicate_columns, axis=1, inplace=True)


In [37]:
df_pca_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7805 entries, 0 to 7804
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   PC1      7805 non-null   float64
 1   PC2      7805 non-null   float64
 2   Cluster  7805 non-null   int32  
dtypes: float64(2), int32(1)
memory usage: 152.6 KB


In [38]:
df_2 = df_pca_predictions.copy()
df_2.insert(0, 'Corporation', first_column) 
br = df.pop('Binary Rating')
df_2.insert(2, 'Binary Rating', br)

df_2.head()

,Corporation,PC1,Binary Rating,PC2,Cluster
0,American States Water Co.,319741.641714,1,801.328232,0
1,Automatic Data Processing Inc.,-728490.657302,1,3519.620476,1
2,Avnet Inc.,-728303.285091,1,1210.233135,1
3,California Water Service Co.,298039.642473,1,807.233946,0
4,Cardinal Health Inc.,-15790.296680,1,1073.498944,0


In [39]:
# Connect to the database
conn = sqlite3.connect('../data/db.sqlite')


df_copy.to_sql('cluster', conn, index=False, if_exists='replace', dtype={'id': 'INTEGER PRIMARY KEY'})
df_2.to_sql('pca', conn, index=False, if_exists='replace', dtype={'id': 'INTEGER PRIMARY KEY'})

conn.close()
